In [73]:
import sys
import os
import math
import pprint
import json
import sqlite3
import dateutil
from dateutil.parser import *
import time
import datetime
from os import listdir
from os.path import isfile, join

In [123]:
#dbpath = "/mnt/cns_storage/home/vorsi/derived"
#dbname = 'githubarchive_full.db'
#datapath = '/mnt/cns_storage/home/github_shared/data/raw'


dbpath = ''
dbname = 'githubarchive_full2.sqlite'
datapath = os.getcwd()


In [124]:
def main():
    build_repository()
    build_push()
    build_watch()
    build_follow()
    build_pulled_request_merged()
    build_pulled_request()
    build_memberadd()
    build_membership()

In [132]:
def build_repository():
    # Create table
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS repository''')
    c.execute('''CREATE TABLE IF NOT EXISTS repository
                 (repositoryid INTEGER PRIMARY KEY, repositoryname TEXT, userlogin TEXT, created_at INTEGER, language TEXT, description TEXT, url TEXT, fork INTEGER)''')
    conn.commit()
    conn.close()

In [133]:
def build_push():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS push''')
    c.execute('''CREATE TABLE IF NOT EXISTS push
                 (pushed_at INTEGER, repositoryid INTEGER, userlogin TEXT)''')
    conn.commit()
    conn.close()

In [134]:
def build_watch():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS watch''')
    c.execute('''CREATE TABLE IF NOT EXISTS watch
                 (watched_at INTEGER, repositoryid INTEGER, byuserlogin TEXT)''')
    conn.commit()
    conn.close()

In [135]:
def build_pulled_request_merged():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS pulled_request_merged''')
    c.execute('''CREATE TABLE IF NOT EXISTS push
                 (merged_at INTEGER, repositoryid INTEGER, userlogin TEXT)''')
    conn.commit()
    conn.close()

In [ ]:
def build_pulled_request():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS pulled_request''')
    c.execute('''CREATE TABLE IF NOT EXISTS push
                 (created_at INTEGER, repositoryid INTEGER, userlogin TEXT)''')
    conn.commit()
    conn.close()

In [136]:
def build_memberadd():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS memberadd''')
    c.execute('''CREATE TABLE IF NOT EXISTS memberadd
                 (added_at INTEGER, repositoryid INTEGER, userlogin TEXT, byuserlogin TEXT)''')
    conn.commit()
    conn.close()

In [ ]:
def build_membership():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS membership''')
    c.execute('''CREATE TABLE IF NOT EXISTS memberadd
                 (added_at TEXT, repositoryid INTEGER, userlogin TEXT, byuserlogin TEXT)''')
    conn.commit()
    conn.close()

In [137]:
def build_follow():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS follow''')
    c.execute('''CREATE TABLE IF NOT EXISTS follow
                 (followed_at INTEGER, targetuserlogin TEXT, byuserlogin TEXT)''')
    conn.commit()
    conn.close()

In [ ]:
def build_fork():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS fork''')
    c.execute('''CREATE TABLE IF NOT EXISTS fork
                 (forked_at INTEGER, targetrepoid INTEGER, byuserlogin TEXT)''')
    conn.commit()
    conn.close()

In [138]:
def build_star():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS star''')
    c.execute('''CREATE TABLE IF NOT EXISTS star
                 (starred_at INTEGER, targetrepoid INTEGER, byuserlogin TEXT)''')
    conn.commit()
    conn.close()

In [139]:
if __name__ == '__main__':
    main()

In [141]:
def date_from_json(creation_date):
    dt = datetime.datetime.strptime(creation_date[:10], '%Y-%m-%d')
    return dt.strftime('%Y-%m-%d')

In [146]:
def date_from_filename(fname):
    dt = datetime.datetime.strptime(fname[len('githubarchive_'):-len('.json')], '%Y-%m-%d')
    return dt.strftime('%Y-%m')

In [142]:
def parse(event, date_f, merging_matter):
    #print event['type']
    if type(event)==dict:
        if 'repo' in event.keys(): 
            #print'k'
            if 'id' in event['repo'].keys():
                #print 'id in repo'
                if merging_matter==False:
                    d=date_f
                    if 'user' in event.keys():
                        if event['user']['type']=='User':
                            u=event['user']['login']
                            p=event['repo']['id']
                            return (u, p, d)
                    elif 'actor' in event.keys():
                        if 'login' in event['actor'].keys():
                            u=event['actor']['login']
                            p=event['repo']['id']
                            return (u, p, d)
                        elif 'payload' in event.keys():
                            if 'actor' in event['payload'].keys():
                                u=event['payload']['actor']
                                p=event['repo']['id']
                                return (u, p, d)
                elif merging_matter==True:
                    d=date_from_json(event['payload']['pull_request']['created_at'][:10])
                    #print d
                    if 'user' in event.keys():
                        #print event.keys()
                        if event['user']['type']=='User':
                            u=event['user']['login']
                            p=event['repo']['id']
                            return (u, p, d)      
                    elif 'actor' in event.keys():
                        u=event['actor']['login']
                        p=event['repo']['id']
                        return (u, p, d)
            else:
                return ('error' ,_,_)
        elif 'repository' in event.keys():
            u=event['actor']
            p=event['repository']['id']
            d=date_f
            return (u, p, d)
        else:
            return ('error' ,_,_)

In [143]:
def parse_follow_event(event):
    if type(event['actor'])==dict:
        f=event['payload']['actor']
        t=event['payload']['target']['login']
    else:
        f=event['actor']
        t=event['payload']['target']['login']
    return (f,t)

In [144]:
def parse_membership_event(event):
    u=event['actor']['login']
    o=event['org']['login']
    a=event['payload']['action']
    return (u,o,a) 

In [145]:
def parse_member_event(event):
    if 'repo' not in event.keys():
        u=event['actor']
        r=event['repository']['name']
        o=event['repository']['owner']
        a=event['payload']['action']
        return (u,r,o,a) 
    elif type(event['payload']['member'])==dict:
        u=event['payload']['member']['login']
        r=event['repo']['name']
        o=event['actor']['login']
        a=event['payload']['action']
    else:
        u=event['payload']['member']
        r=event['repo']['name']
        o=event['payload']['actor']
        a=event['payload']['action']
        return (u,r,o,a)         

In [ ]:
def parse_create_event(event):
    if event['payload'].get('object')=='repository' and event['repo'].get('id'):
        repo_name=event['payload'].get('name')
        repoid=event['repo']['id']
        created_at=date_from_json(event.get('created_at'))
        user=event['actor']['login']
        description=''
        return repo_name, repoid, created_at, user, description
    elif event.get('payload') and event['payload'].get('ref_type')=='repository' and event.get('repository'):
        repo_name=event['repository']['name']
        repoid=event['repository']['id']
        created_at=date_from_json(event['repository'].get('created_at'))
        user=event['actor']
        description=event['payload'].get('description')
        return repo_name, repoid, created_at, user, description

In [ ]:
def all_event_parser(event, date_file):
    et=event['type']
    if et=='CreateEvent':
        query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?,?,?,?)'
        c.execute(query, (jsonline["repository"].get("id"), jsonline["repository"].get("name"), jsonline["actor_attributes"].get("gravatar_id"), jsonline["repository"]["created_at"], jsonline["repository"].get("language"), jsonline["repository"].get("description"), jsonline["repository"].get("url"), jsonline["repository"].get("fork")))
        i = i+1
    elif et=='FollowEvent':
        f,t=parse_follow_event(event)
        follow_follower_network.append((f,t,date_file))
        users.add(f)
        users.add(t)
    elif et=='MembershipEvent':
        u,o,a=parse_membership_event(event)
        memberships.append((u,o,a, date_file))
        users.add(u)
    elif et=='MemberEvent':
        u,r,o,a=parse_member_event(event)
        repo_memberships.append((u,r,o,a,date_file))
        users.add(u)
        users.add(o)
    elif et=='PullRequestEvent' and event['payload']['action']=='closed':
        if 'merged' in event ['payload']['pull_request']:
            if event['payload']['pull_request']['merged']==True:
                user, project, date=parse(event, date_file, True)
                all_projects=get_projects_dict(all_projects, user, project, date, u'Closed_PullRequestEvent')
                user_activitiy=get_user_activity(user_activity, user, project, date, et)
                #return all_projects, follow_follower_network, memberships, repo_memberships, users
    elif et not in ['GistEvent','GollumEvent','DeleteEvent','MemberEvent']:
        user, project, date=parse(event, date_file, False)
        if user!='error':
            all_projects=get_projects_dict(all_projects, user, project, date, et)
            user_activity=get_user_activity(user_activity, user, project, date, et)
    return all_projects, follow_follower_network, memberships, repo_memberships, users,user_activity

In [140]:
#filenames = [ f for f in listdir(datapath) if isfile(join(datapath,f)) ]
filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_2011") and i.endswith('json')])

In [130]:
for f in filenames:
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    i = 0
    with open(os.path.join(dbpath,f), 'r') as myfile:
            data=myfile.read()
            date_file=date_from_filename(f)
            lines=data.splitlines()
            for i in range(0,len(lines)):
                if i%10000==0:
                    print len(lines), i
            try:
                jsonline=json.loads(unicode(lines[i], errors='ignore').strip())
                et=jsonline['type']
                if jsonline.get("type") == "CreateEvent"and jsonline.get("repository") and jsonline.get("payload") and jsonline.get("actor_attributes") and jsonline["payload"].get("ref_type") == "repository" and jsonline["repository"].get("id"): # we need to have the id
                    # Time format to be parsed: 2012-10-26T23:23:47-07:00
                    jsonline["repository"]["created_at"] = int(round(time.mktime(dateutil.parser.parse(jsonline["repository"]["created_at"]).timetuple())))
                    query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?,?,?,?)'
                    c.execute(query, (jsonline["repository"].get("id"), jsonline["repository"].get("name"), jsonline["actor_attributes"].get("gravatar_id"), jsonline["repository"]["created_at"], jsonline["repository"].get("language"), jsonline["repository"].get("description"), jsonline["repository"].get("url"), jsonline["repository"].get("fork")))
                    i = i+1
            except ValueError:
                for k in ['{"repo"'+line for line in lines[i].split('{"repo"')[1:]]:
                    jsonline=json.loads(k)
                    et=jsonline['type']
                    if jsonline.get("type") == "CreateEvent" and jsonline.get("repository") and jsonline.get("payload") and jsonline.get("actor_attributes") and jsonline["payload"].get("ref_type") == "repository" and jsonline["repository"].get("id"): # we need to have the id
                        # Time format to be parsed: 2012-10-26T23:23:47-07:00
                        jsonline["repository"]["created_at"] = int(round(time.mktime(dateutil.parser.parse(jsonline["repository"]["created_at"]).timetuple())))
                        query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?,?,?,?)'
                        c.execute(query, (jsonline["repository"].get("id"), jsonline["repository"].get("name"), jsonline["actor_attributes"].get("gravatar_id"), jsonline["repository"]["created_at"], jsonline["repository"].get("language"), jsonline["repository"].get("description"), jsonline["repository"].get("url"), jsonline["repository"].get("fork")))
                        i = i+1
            print "Done file " + filename + ". Found and inserted " + str(i) + " valid CreationEvents."
            conn.commit()
            conn.close()

32197 0
32197 10000
32197 20000
32197 30000
Done file githubarchive_2013-02-27.json. Found and inserted 32196 valid CreationEvents.
39281 0
39281 10000
39281 20000
39281 30000
Done file githubarchive_2013-02-27.json. Found and inserted 39280 valid CreationEvents.
44180 0
44180 10000
44180 20000
44180 30000
44180 40000
Done file githubarchive_2013-02-27.json. Found and inserted 44179 valid CreationEvents.
49698 0
49698 10000
49698 20000
49698 30000
49698 40000
Done file githubarchive_2013-02-27.json. Found and inserted 49697 valid CreationEvents.
47410 0
47410 10000
47410 20000
47410 30000
47410 40000
Done file githubarchive_2013-02-27.json. Found and inserted 47409 valid CreationEvents.
49694 0
49694 10000
49694 20000
49694 30000
49694 40000
Done file githubarchive_2013-02-27.json. Found and inserted 49693 valid CreationEvents.
56858 0
56858 10000
56858 20000
56858 30000
56858 40000
56858 50000
Done file githubarchive_2013-02-27.json. Found and inserted 56857 valid CreationEvents.
5103

In [120]:
for f in filenames:
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    i = 0
    with open(os.path.join(dbpath,f), 'r') as myfile:
            data=myfile.read()
            lines=data.splitlines()
            for i in range(0,len(lines)):
                if i%10000==0:
                    print len(lines), i
            try:
                jsonline=json.loads(unicode(lines[i], errors='ignore').strip())
                if jsonline.get("type") == "CreateEvent"and jsonline.get("repository") and jsonline.get("payload") and jsonline.get("actor_attributes") and jsonline["payload"].get("ref_type") == "repository" and jsonline["repository"].get("id"): # we need to have the id
                    # Time format to be parsed: 2012-10-26T23:23:47-07:00
                    jsonline["repository"]["created_at"] = int(round(time.mktime(dateutil.parser.parse(jsonline["repository"]["created_at"]).timetuple())))
                    query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?,?,?,?)'
                    c.execute(query, (jsonline["repository"].get("id"), jsonline["repository"].get("name"), jsonline["actor_attributes"].get("gravatar_id"), jsonline["repository"]["created_at"], jsonline["repository"].get("language"), jsonline["repository"].get("description"), jsonline["repository"].get("url"), jsonline["repository"].get("fork")))
                    i = i+1
            except ValueError:
                for k in ['{"repo"'+line for line in lines[i].split('{"repo"')[1:]]:
                    jsonline=json.loads(k)
                    if jsonline.get("type") == "CreateEvent" and jsonline.get("repository") and jsonline.get("payload") and jsonline.get("actor_attributes") and jsonline["payload"].get("ref_type") == "repository" and jsonline["repository"].get("id"): # we need to have the id
                        # Time format to be parsed: 2012-10-26T23:23:47-07:00
                        jsonline["repository"]["created_at"] = int(round(time.mktime(dateutil.parser.parse(jsonline["repository"]["created_at"]).timetuple())))
                        query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?,?,?,?)'
                        c.execute(query, (jsonline["repository"].get("id"), jsonline["repository"].get("name"), jsonline["actor_attributes"].get("gravatar_id"), jsonline["repository"]["created_at"], jsonline["repository"].get("language"), jsonline["repository"].get("description"), jsonline["repository"].get("url"), jsonline["repository"].get("fork")))
                        i = i+1
            print "Done file " + filename + ". Found and inserted " + str(i) + " valid CreationEvents."
            conn.commit()
            conn.close()

{u'actor': {u'avatar_url': u'https://secure.gravatar.com/avatar/5df5f29da54fbebf5d0318e7aa772fce?d=http://github.dev%2Fimages%2Fgravatars%2Fgravatar-user-420.png',
  u'gravatar_id': u'5df5f29da54fbebf5d0318e7aa772fce',
  u'id': 175425,
  u'login': u'oincex',
  u'url': u'https://api.github.dev/users/oincex'},
 u'created_at': u'2011-04-08T23:59:51Z',
 u'id': u'1240963541',
 u'payload': {u'name': u'oincex',
  u'object': u'repository',
  u'object_name': None},
 u'public': True,
 u'repo': {u'name': u'/', u'url': u'https://api.github.dev/repos//'},
 u'type': u'CreateEvent'}

In [82]:
!ls

2014-03_all_projects.json           githubarchive_2011-02-14.json
2014-03_follow_follower_network.csv githubarchive_2011-03-14.json
2014-03_memberships.csv             githubarchive_2011-03-15.json
2014-03_repo_memberships.csv        githubarchive_2011-03-28.json
2014-03_user_activity.json          githubarchive_2011-03-30.json
2014-03_users.csv                   githubarchive_2011-04-06.json
Analyse_users.ipynb                 githubarchive_2011-04-08.json
Untitled.ipynb                      githubarchive_2013-02-27.json
clean_code.py                       githubarchive_2014-03-04.json
create_db.ipynb                     githubarchive_2015-12-29.json
database_creation.ipynb             githubarchive_full.db
database_creation.py                githubarchive_full.sqlite
date_id_repos.json                  githubarchivedownloader.py
diversity-master                    learn_sql.ipynb
emailtome.py                        names.json
emailtome.pyc                       new_code_for_aggregati

In [83]:
! cat githubarchive_full

cat: githubarchive_full: No such file or directory
